Code Author -
@ Naman_Jaswani

In [1]:
import numpy as np
import pandas as pd
import json
import re
import ast

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

from tqdm.notebook import tqdm
import gc

import matplotlib.pyplot as plt

# Model
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

import os

import warnings
warnings.filterwarnings('ignore')


In [2]:
print(tf.__version__)
print(keras.__version__)

2.4.1
2.4.3


In [3]:
top = pd.read_pickle('../input/march-6-complete-tagged/TOP WEAR.pickle')
bottom = pd.read_pickle('../input/march-6-complete-tagged/BOTTOM WEAR.pickle')
foot = pd.read_pickle('../input/march-6-complete-tagged/FOOT WEAR.pickle')
full = pd.read_pickle('../input/march-6-complete-tagged/FULL WEAR.pickle')
acc = pd.read_pickle('../input/march-6-complete-tagged/ACCESSORIES.pickle')

In [4]:
lst = [top, bottom, foot, full, acc]
data = pd.concat(lst)
data = data.reset_index(drop=True)
data.shape

(311716, 5)

In [5]:
del top
del bottom
del foot
del full
del acc

In [6]:
class CFG:
    BOOL_REMOVE_TAGS=True
    REMOVE_TAGS = ['sunglass_bridge_material',
              'watch_case_bezel_material',
              'jewellery_set_metal_color',
              'jewellery_set_emerald_color',
              'sunglass_lens_color',
              'jewellery_set_pearl_color',
              'jewellery_set_ruby_color',
              'jewellery_set_sapphire_color',
              'jewellery_set_silver_color',
              'jewellery_set_gemstone',
              'trouser_occasion',
              'jean_bottomwear_length_type',
              'gown_belt_material',
              'sari_border_length',
              'gown_bust_size',
              'lining material',
              'detail_placement'
              ]
    
    BATCH_SIZE = 16
    EPOCHS = 5
    MAX_LEN=90
    NUM_SAMPLES=20000
    
        

In [7]:
data['Category'].unique()

array(['Top wear', 'Bottom Wear', 'Foot Wear', 'Full wear', 'Accessories'],
      dtype=object)

In [8]:
for cat in ['Top wear', 'Bottom Wear', 'Full wear', 'Foot Wear', 'Accessories']:
    print(cat, data[data['Category']==cat].shape)

Top wear (145327, 5)
Bottom Wear (25729, 5)
Full wear (35835, 5)
Foot Wear (46100, 5)
Accessories (58725, 5)


In [9]:
n_samples = CFG.NUM_SAMPLES

In [10]:
data_top = data[data['Category']=='Top wear']
# data_top = data_top.sample(frac=1).reset_index(drop=True)
if(n_samples < data_top.shape[0]):
    data_top = data_top[:n_samples]

data_bottom = data[data['Category']=='Bottom Wear']
# data_bottom = data_bottom.sample(frac=1).reset_index(drop=True)
if(n_samples < data_bottom.shape[0]):
    data_bottom = data_bottom[:n_samples]


data_foot = data[data['Category']=='Foot Wear']
# data_foot = data_foot.sample(frac=1).reset_index(drop=True)
if(n_samples < data_foot.shape[0]):
    data_foot = data_foot[:n_samples]



data_full = data[data['Category']=='Full wear']
# data_full = data_full.sample(frac=1).reset_index(drop=True)
if(n_samples < data_full.shape[0]):
    data_full = data_full[:n_samples]


data_acc = data[data['Category']=='Accessories']
# data_acc = data_acc.sample(frac=1).reset_index(drop=True)
if(n_samples < data_acc.shape[0]):
    data_acc = data_acc[:n_samples]



data = data_top.append([data_bottom, data_foot, data_full, data_acc])
del data_bottom
del data_foot
del data_full
del data_acc

gc.collect()

0

In [11]:
data = data.reset_index(drop=True)
data

,Unnamed: 0,Category,vertical,Descriptions,Tag
0,1,Top wear,t_shirt,"GAP's staples have the perfect slouchy, relaxe...","[B_brand, O, O, O, O, O, O, O, B_fit, I_fit, O..."
1,2,Top wear,t_shirt,"WILLIAM TEE,XL","[O, O, O, B_size]"
2,3,Top wear,t_shirt,"Grey Melange printed T-shirt, has a round neck...","[B_color, B_brand, B_pattern, O, O, O, O, B_co..."
3,4,Top wear,t_shirt,Gear up for a refreshing jogging session and r...,"[O, O, O, O, O, O, O, O, O, O, O, O, B_fit, O,..."
4,5,Top wear,t_shirt,"Burgundy, white and black colourblocked T-shir...","[O, O, B_color, O, B_color, O, O, O, O, O, B_c..."
...,...,...,...,...,...
99995,273001,Accessories,watch,Display Type: Analogue; Movement Type: Quartz:...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
99996,273002,Accessories,watch,Display Type: Analogue; Movement Type: Quartz:...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, B_colo..."
99997,273003,Accessories,watch,Display Type: Analog; Movement Type: Quartz ; ...,"[O, O, O, O, O, O, O, O, O, O, B_closure, O, O..."
99998,273004,Accessories,watch,Display Type: Analogue; Movement Type: Quartz:...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, B_colo..."


In [12]:
if CFG.BOOL_REMOVE_TAGS:

    r_tags = []
    for t in CFG.REMOVE_TAGS:
        r_tags.append(f'B_{t}')
        r_tags.append(f'I_{t}')    

    for i in tqdm(range(data.shape[0])):
        l = data['Tag'][i]
        for j in range(len(l)):
            if(l[j] in r_tags):
    #             print('yes')
                l[j] = 'O'
        data['Tag'][i] = l

  0%|          | 0/100000 [00:00<?, ?it/s]

In [13]:
data['Descriptions'][0]

"GAP's staples have the perfect slouchy, relaxed fit like this tee. It has a V-neck, short sleeves and a chest patch pocket. This versatile white piece works equally well with skinny jeans or tailoring."

# Test Dataset

In [14]:
test = pd.read_csv('../input/new-test-files/test.csv')
test_output = pd.read_csv('../input/new-test-files/test_output.csv')
test.shape

(22421, 4)

# BiLSTM data

In [15]:
dataset = pd.DataFrame()
lst = []

for i in tqdm(range(data.shape[0])):

    dcrp = data['Descriptions'][i]
    dcrp = word_tokenize(dcrp.lower())
    num = [f'Description_{i}']* len(dcrp)  
    cat = [data['Category'][i]]*len(dcrp)
    
    tags = data['Tag'][i]
    dct = pd.DataFrame(list(zip(cat, num, dcrp, tags)), columns=['Category','# Description','word', 'tag'])
    lst.append(dct)
    
dataset = pd.concat(lst)
dataset = dataset.reset_index(drop=True)
dataset.shape



  0%|          | 0/100000 [00:00<?, ?it/s]

(5868634, 4)

In [16]:
test_dataset = pd.DataFrame()
lst = []

for i in tqdm(range(test.shape[0])):

    dcrp = ast.literal_eval(test['Words'][i])
    dcrp_ = []
    for x in dcrp:
        dcrp_.append(x.lower())
        
    num = [f'Description_{i}']* len(dcrp_)  
    cat = [test['Category'][i]]*len(dcrp_)
    
#     tags = test['Tag'][i]
    dct = pd.DataFrame(list(zip(cat, num, dcrp_)), columns=['Category','# Description','word'])
    lst.append(dct)
    
test_dataset = pd.concat(lst)
test_dataset = test_dataset.reset_index(drop=True)
test_dataset.head()


  0%|          | 0/22421 [00:00<?, ?it/s]

,Category,# Description,word
0,Top Wear,Description_0,own
1,Top Wear,Description_0,a
2,Top Wear,Description_0,unique
3,Top Wear,Description_0,piece
4,Top Wear,Description_0,from


In [17]:
# of unique tags
dataset['tag'].nunique()

100

In [18]:
# Unique words, Unique tags

words = list(set(list(dataset['word'].values)+list(test_dataset['word'].values)))
words.append('ENDPAD')
tags = list(set(dataset['tag'].values))

In [19]:
num_words = len(words)
num_tags = len(tags)

In [20]:
class Sentence_getter(object):
    def __init__(self, data):
        self.data = data
        agg_fun = lambda s: [(w, t) for w,t in zip(s['word'].values.tolist(),
                                                  s['tag'].values.tolist())]

        self.grouped = self.data.groupby('# Description').apply(agg_fun)
        
        self.sentences = [i for i in self.grouped]

In [21]:
class Test_Sentence_getter(object):
    def __init__(self, data):
        self.data = data
        agg_fun = lambda s: [w for w in s['word'].values.tolist()]
                                                  
        self.grouped = self.data.groupby('# Description').apply(agg_fun)
        
        self.sentences = [self.grouped[f'Description_{i}'] for i in range(len(self.grouped))]

In [22]:
getter= Sentence_getter(dataset)
sentences = getter.sentences

test_getter= Test_Sentence_getter(test_dataset)
test_sentences = test_getter.sentences

# print(test_sentences[3])

del getter
del test_getter


In [23]:
# Vocabulary
word2idx = {w : i+1 for i,w in enumerate(words)}
tag2idx = {t: i for i,t in enumerate(tags)}

In [24]:
import gc
gc.collect()

40

In [25]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

max_len = CFG.MAX_LEN

X = [[word2idx[row[0]] for row in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding='post', value=num_words-1, truncating='post')

x_test = [[word2idx[row] for row in s] for s in test_sentences]
x_test = pad_sequences(maxlen=max_len, sequences=x_test, padding='post', value=num_words-1, truncating='post')

y = [[tag2idx[row[1]] for row in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding='post', value=tag2idx['O'], truncating='post')
y = [to_categorical(i, num_classes=num_tags) for i in y]


In [26]:
del sentences
del test_sentences
gc.collect()

20

In [27]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X, np.array(y), test_size=0.2, shuffle=True, random_state=42)

In [28]:
# Detect TPU
try:
    
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
   
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [29]:
from keras.optimizers import Nadam

In [30]:
with strategy.scope():
    input_word = Input(shape=(max_len,))
    model = Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len)(input_word)

    model = SpatialDropout1D(0.1)(model)
    model = Bidirectional(LSTM(units=max_len, return_sequences=True, recurrent_dropout=0.1))(model)

    out = TimeDistributed(Dense(num_tags, activation='softmax'))(model)

    model = Model(input_word, out)
    model.compile(optimizer=Nadam(), loss='categorical_crossentropy', metrics='accuracy')
model.summary()



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 90)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 90, 90)            5981940   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 90, 90)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 90, 180)           130320    
_________________________________________________________________
time_distributed (TimeDistri (None, 90, 100)           18100     
Total params: 6,130,360
Trainable params: 6,130,360
Non-trainable params: 0
_________________________________________________________________


In [31]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, verbose=0, mode='max', restore_best_weights=False)

In [32]:
callbacks=[early_stopping]

history = model.fit(x_train[:x_train.shape[0]//2], y_train[:y_train.shape[0]//2],
                    validation_data=(x_val, y_val), 
                    batch_size=CFG.BATCH_SIZE,
                    epochs=CFG.EPOCHS,
                    verbose=1,
                    callbacks=callbacks,
#                     steps_per_epoch = X.shape[0] // 16
                   )

Epoch 1/5
2500/2500 [==============================] - 79s 28ms/step - loss: 0.3179 - accuracy: 0.9548 - val_loss: 0.0310 - val_accuracy: 0.9913
Epoch 2/5
2500/2500 [==============================] - 64s 26ms/step - loss: 0.0249 - accuracy: 0.9928 - val_loss: 0.0165 - val_accuracy: 0.9951
Epoch 3/5
2500/2500 [==============================] - 64s 26ms/step - loss: 0.0126 - accuracy: 0.9963 - val_loss: 0.0122 - val_accuracy: 0.9964
Epoch 4/5
2500/2500 [==============================] - 64s 25ms/step - loss: 0.0076 - accuracy: 0.9978 - val_loss: 0.0089 - val_accuracy: 0.9975
Epoch 5/5
2500/2500 [==============================] - 63s 25ms/step - loss: 0.0052 - accuracy: 0.9985 - val_loss: 0.0083 - val_accuracy: 0.9977


In [33]:
history = model.fit(x_train[x_train.shape[0]//2 : ], y_train[y_train.shape[0]//2 : ],
                    validation_data=(x_val, y_val), 
                    batch_size=CFG.BATCH_SIZE,
                    epochs=CFG.EPOCHS,
                    verbose=1,
                    callbacks=callbacks,
#                     steps_per_epoch = X.shape[0] // 16
                   )

Epoch 1/5
2500/2500 [==============================] - 68s 27ms/step - loss: 0.0074 - accuracy: 0.9979 - val_loss: 0.0060 - val_accuracy: 0.9983
Epoch 2/5
2500/2500 [==============================] - 66s 26ms/step - loss: 0.0044 - accuracy: 0.9988 - val_loss: 0.0052 - val_accuracy: 0.9986
Epoch 3/5
2500/2500 [==============================] - 65s 26ms/step - loss: 0.0029 - accuracy: 0.9992 - val_loss: 0.0049 - val_accuracy: 0.9986
Epoch 4/5
2500/2500 [==============================] - 66s 26ms/step - loss: 0.0021 - accuracy: 0.9994 - val_loss: 0.0052 - val_accuracy: 0.9986
Epoch 5/5
2500/2500 [==============================] - 65s 26ms/step - loss: 0.0016 - accuracy: 0.9996 - val_loss: 0.0049 - val_accuracy: 0.9987


In [34]:
model.save('BILSTM_model_23.h5')

# Inference on a random sample from test_output

In [35]:
i = np.random.randint(0, test_output.shape[0])
idx = test_output['Id'][i]

ii = test[test['Id']==idx].index

p = model.predict(np.array([x_test[ii[0]]]))

p = np.argmax(p, axis=-1)

y_true = ast.literal_eval(test_output['Tags'][i])


print("{:15}{}\t{}\n".format('Word', 'Pred', 'True'))
print("-"*30)

for(w, pred, t) in zip(x_test[ii[0]], p[0], y_true):
    print("{:15}{}\t{}\t".format(words[w-1], tags[pred], t))

Word           Pred	True

------------------------------
revin          O	B_BRAND	
lite           O	B_COLOR	
green          B_color	I_COLOR	
button         B_closure	O	
model          O	O	
multi          O	B_COLOR	
colour         O	I_COLOR	
tshirt         O	O	
.              O	O	
it             O	O	
comes          O	O	
with           O	O	
m              B_size	B_SIZE	
size           O	O	
and            O	O	
can            O	O	
be             O	O	
weared         O	O	
for            O	O	
all            O	O	
type           O	O	
of             O	O	
occassions     O	O	
.              O	O	


## Scores on test_output

In [36]:
test_preds = []
test_trues = []

for i in tqdm(range(test_output.shape[0])):
    idx = test_output['Id'][i]

    ii = test[test['Id']==idx].index
#     print(ii[0])

    p = model.predict(np.array([x_test[ii[0]]]))
    p = np.argmax(p, axis=-1)
    
    pp = []
    for p_ in p[0]:
        pp.append(tags[p_])
    true = ast.literal_eval(test_output['Tags'][i])
    
    
    test_preds.append(pp)
    test_trues.append(true)
    
test_preds_new = []
i = 0
for p in tqdm(test_preds):
    lst = [x.upper() for x in p]
    l = []
    cnt = 0
    for x in lst:
        if(cnt >= len(ast.literal_eval(test_output['Words'][i]))):
            break
        l.append(x)
        cnt+=1
    length = len(l)
    if(length < len(ast.literal_eval(test_output['Words'][i]))):
        l += ['O'] * (len(ast.literal_eval(test_output['Words'][i]))-length)
    i += 1
    test_preds_new.append(l)


  0%|          | 0/112 [00:00<?, ?it/s]

  0%|          | 0/112 [00:00<?, ?it/s]

In [37]:
!pip install seqeval

     |████████████████████████████████| 43 kB 494 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=0b2102f0a216c50e1cf4a1dce711c6dcd2d5d1291501878eb832ebbe80cc9190
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [38]:
import seqeval
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report


print(classification_report(test_trues, test_preds_new[:len(test_trues)]))

                              precision    recall  f1-score   support

                      _BRAND       0.47      0.12      0.19        58
                    _CLOSURE       0.06      0.17      0.09         6
                     _COLLAR       0.88      0.59      0.71        49
                      _COLOR       0.75      0.75      0.75        64
                _COLOR_SHADE       0.00      0.00      0.00         0
                 _DIAL_COLOR       1.00      1.00      1.00         2
  _DRESS_TOPWEAR_LENGTH_TYPE       0.00      0.00      0.00         1
                     _FABRIC       0.82      0.21      0.33        68
                _FABRIC_CARE       0.50      0.12      0.20        33
              _FABRIC_PURITY       0.00      0.00      0.00         0
                        _FIT       0.19      0.45      0.27        11
 _FOOTWEAR_PATTERN_PLACEMENT       0.00      0.00      0.00         0
                _GOWN_DESIGN       0.00      0.00      0.00         0
            _GOWN_W

In [39]:
preds = []

for i in tqdm(range(x_test.shape[0])):
    p = model.predict(np.array([x_test[i]]))
    p = np.argmax(p, axis=-1)
    
    lst = [tags[x] for x in p[0]]
    preds.append(lst)
    
    
    

preds_new = []
i = 0
for p in tqdm(preds):
    lst = [x.upper() for x in p]
    l = []
    cnt = 0
    for x in lst:
        if(cnt >= len(ast.literal_eval(test['Words'][i]))):
            break
        l.append(x)
        cnt+=1
    length = len(l)
    if(length < len(ast.literal_eval(test['Words'][i]))):
        l += ['O'] * (len(ast.literal_eval(test['Words'][i]))-length)
    i += 1
    preds_new.append(l)

  0%|          | 0/22421 [00:00<?, ?it/s]

  0%|          | 0/22421 [00:00<?, ?it/s]

In [40]:
test['Tags'] = preds_new
test.to_csv('Submission.csv', index='False')
test.head()

,Unnamed: 0,Id,Words,Category,Tags
0,0,0cbabaee2-759e-11eb-8e68-02010a3042d3,"['Own', 'a', 'unique', 'piece', 'from', 'the',...",Top Wear,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,1,1cbabb108-759e-11eb-8e68-02010a3042d3,"['White', 'T', 'shirt', ',', 'has', 'a', 'V', ...",Top Wear,"[B_COLOR, O, O, O, O, O, B_COLLAR, I_COLLAR, O..."
2,2,2cbabb1bc-759e-11eb-8e68-02010a3042d3,"['Cotton', 'Printed', '3', 'Pcs', 'Assorted', ...",Top Wear,"[O, B_PATTERN, O, O, O, O, O, B_COLLAR, I_COLL..."
3,3,3cbabb252-759e-11eb-8e68-02010a3042d3,"['Designed', 'in', 'an', 'appealing', 'shade',...",Top Wear,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,4,4cbabb2de-759e-11eb-8e68-02010a3042d3,"['Symbols', 'of', 'luck', ',', 'intelligence',...",Top Wear,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
